In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from keras.models import load_model
import time #to calculate the computation time

2022-10-13 02:01:10.223480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-13 02:01:10.223555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# specifying constant parameters
#input varioable: Q, C_A0
#state variable: T, C_A

T_0 = 300
V = 1
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000
F = 5
E = 5*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314

C_A0s = 4   # the steady state for input variable C_A0
Q_s = 0.0  # the steady state for input variable Q

C_As = 1.95 # the steady state for state variable C_A
T_s = 402  # the steady state for state variable T

t_final = 0.01  #the control period
t_step = 1e-4   # the step to use first-principle to calculate the state
P = np.array([[1060, 22], [22, 0.52]])

In [3]:
# generating inputs and initial states for CSTR, all expressed in deviation form

# Test 1 full stability region
u1_list = np.linspace(-3.5, 3.5, 20, endpoint=True)
u2_list = np.linspace(-5e5, 5e5, 20, endpoint=True)
T_initial = np.linspace(300, 550, 20, endpoint=True) - T_s
CA_initial = np.linspace(1, 4., 20, endpoint=True) - C_As    # CA and T must be >0
#print(T_initial)
#control variable: C_A0, Q
#state variable: C_A, T

# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:   
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 368:   #calculate the stability region for state 
            CA_start.append(CA)
            T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
print(CA_start.shape)
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))
print(x_deviation.shape)  # the initial state is in 

number of initial conditions: 60
(1, 60)
shape of x_deviation is (60, 2)
(60, 2)


In [4]:
def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """
    
    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time
    
    C_A = C_A_initial + C_As  # the real state.the derivation plus the steady state
    T = T_initial + T_s
    
    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
        
        C_A += dCAdt * t_step
        T += dTdt * t_step
        
        if i%5 ==0:
            C_A_list.append(C_A - C_As)  # in deviation form
            T_list.append(T - T_s)  # in deviation form 
    
    return C_A_list, T_list

In [5]:
# get X and y data for training and testing

CA_output = list()
T_output = list()

CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()   #input variable for 

for u1 in u1_list:
    C_A0 = u1 + C_A0s
    
    for u2 in u2_list:
        Q = u2 + Q_s
        
        for C_A_initial, T_initial in x_deviation:
            CA0_input.append(u1)
            Q_input.append(u2)
            CA_input.append(C_A_initial)
            T_input.append(T_initial)
            
            C_A_list, T_list = CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial)
            CA_output.append(C_A_list)
            T_output.append(T_list)

In [6]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1,1)

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1,1)

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1,1)

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1,1)

RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)   #the value for input variable and the initial value for state variable 

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step, not sure if my treatment here is correct
"""
print("RNN_input shape is {}".format(RNN_input.shape))
RNN_input = RNN_input.repeat(20, axis=1)  # to keep consensus with the shape for RNN_output, since the output variable is collected 100(0.01/1e-4) times for each RNN_input
print("RNN_input shape is {}".format(RNN_input.shape))

RNN_input shape is (24000, 1, 4)
RNN_input shape is (24000, 20, 4)


In [7]:
# collate output for RNN

CA_output = np.array(CA_output)
CA_output = CA_output.reshape(-1, 20, 1)

T_output = np.array(T_output)
T_output = T_output.reshape(-1, 20, 1)

RNN_output = np.concatenate((T_output, CA_output), axis=2)
print("RNN_output shape is {}".format(RNN_output.shape))  # output shape: number of samples x timestep x variables

RNN_output shape is (24000, 20, 2)


In [8]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.3, random_state=123)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.StandardScaler().fit(X_train.reshape(-1, 4))
scaler_y = preprocessing.StandardScaler().fit(y_train.reshape(-1, 2))


X_train = scaler_X.transform(X_train.reshape(-1, 4)).reshape(-1,20,4)
X_test = scaler_X.transform(X_test.reshape(-1, 4)).reshape(-1,20,4)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,20,2)
y_test = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,20,2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16800, 20, 4) (7200, 20, 4) (16800, 20, 2) (7200, 20, 2)


In [22]:
model = Sequential()
model.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model.add(Dense(2, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

t0 = time.time()
history = model.fit(X_train, y_train, epochs=300, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()
print(t1-t0)

Epoch 1/300
50/50 - 2s - loss: 0.3592 - mse: 0.3592 - val_loss: 0.0680 - val_mse: 0.0680 - 2s/epoch - 35ms/step
Epoch 2/300
50/50 - 1s - loss: 0.0451 - mse: 0.0451 - val_loss: 0.0312 - val_mse: 0.0312 - 596ms/epoch - 12ms/step
Epoch 3/300
50/50 - 1s - loss: 0.0245 - mse: 0.0245 - val_loss: 0.0202 - val_mse: 0.0202 - 530ms/epoch - 11ms/step
Epoch 4/300
50/50 - 1s - loss: 0.0171 - mse: 0.0171 - val_loss: 0.0152 - val_mse: 0.0152 - 621ms/epoch - 12ms/step
Epoch 5/300
50/50 - 1s - loss: 0.0134 - mse: 0.0134 - val_loss: 0.0124 - val_mse: 0.0124 - 537ms/epoch - 11ms/step
Epoch 6/300
50/50 - 1s - loss: 0.0112 - mse: 0.0112 - val_loss: 0.0105 - val_mse: 0.0105 - 597ms/epoch - 12ms/step
Epoch 7/300
50/50 - 1s - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0092 - val_mse: 0.0092 - 560ms/epoch - 11ms/step
Epoch 8/300
50/50 - 1s - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0081 - val_mse: 0.0081 - 570ms/epoch - 11ms/step
Epoch 9/300
50/50 - 1s - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0072 - val_mse: 0

Epoch 68/300
50/50 - 1s - loss: 4.3482e-04 - mse: 4.3482e-04 - val_loss: 3.5962e-04 - val_mse: 3.5962e-04 - 619ms/epoch - 12ms/step
Epoch 69/300
50/50 - 1s - loss: 3.5899e-04 - mse: 3.5899e-04 - val_loss: 3.3618e-04 - val_mse: 3.3618e-04 - 619ms/epoch - 12ms/step
Epoch 70/300
50/50 - 1s - loss: 4.3452e-04 - mse: 4.3452e-04 - val_loss: 4.9366e-04 - val_mse: 4.9366e-04 - 593ms/epoch - 12ms/step
Epoch 71/300
50/50 - 1s - loss: 3.8040e-04 - mse: 3.8040e-04 - val_loss: 4.6170e-04 - val_mse: 4.6170e-04 - 542ms/epoch - 11ms/step
Epoch 72/300
50/50 - 1s - loss: 3.4917e-04 - mse: 3.4917e-04 - val_loss: 3.7143e-04 - val_mse: 3.7143e-04 - 617ms/epoch - 12ms/step
Epoch 73/300
50/50 - 1s - loss: 3.5474e-04 - mse: 3.5474e-04 - val_loss: 3.6677e-04 - val_mse: 3.6677e-04 - 705ms/epoch - 14ms/step
Epoch 74/300
50/50 - 1s - loss: 3.5663e-04 - mse: 3.5663e-04 - val_loss: 3.4940e-04 - val_mse: 3.4940e-04 - 616ms/epoch - 12ms/step
Epoch 75/300
50/50 - 1s - loss: 3.3146e-04 - mse: 3.3146e-04 - val_loss: 2.9

Epoch 130/300
50/50 - 1s - loss: 1.3901e-04 - mse: 1.3901e-04 - val_loss: 2.0206e-04 - val_mse: 2.0206e-04 - 582ms/epoch - 12ms/step
Epoch 131/300
50/50 - 1s - loss: 1.3827e-04 - mse: 1.3827e-04 - val_loss: 2.2163e-04 - val_mse: 2.2163e-04 - 542ms/epoch - 11ms/step
Epoch 132/300
50/50 - 1s - loss: 1.4685e-04 - mse: 1.4685e-04 - val_loss: 1.4497e-04 - val_mse: 1.4497e-04 - 536ms/epoch - 11ms/step
Epoch 133/300
50/50 - 1s - loss: 1.5908e-04 - mse: 1.5908e-04 - val_loss: 1.0827e-04 - val_mse: 1.0827e-04 - 607ms/epoch - 12ms/step
Epoch 134/300
50/50 - 1s - loss: 1.6144e-04 - mse: 1.6144e-04 - val_loss: 1.2348e-04 - val_mse: 1.2348e-04 - 532ms/epoch - 11ms/step
Epoch 135/300
50/50 - 1s - loss: 1.4175e-04 - mse: 1.4175e-04 - val_loss: 1.5723e-04 - val_mse: 1.5723e-04 - 597ms/epoch - 12ms/step
Epoch 136/300
50/50 - 1s - loss: 1.2924e-04 - mse: 1.2924e-04 - val_loss: 1.1999e-04 - val_mse: 1.1999e-04 - 558ms/epoch - 11ms/step
Epoch 137/300
50/50 - 1s - loss: 1.2524e-04 - mse: 1.2524e-04 - val_l

Epoch 192/300
50/50 - 1s - loss: 1.1516e-04 - mse: 1.1516e-04 - val_loss: 6.6742e-05 - val_mse: 6.6742e-05 - 661ms/epoch - 13ms/step
Epoch 193/300
50/50 - 1s - loss: 6.0152e-05 - mse: 6.0152e-05 - val_loss: 8.8376e-05 - val_mse: 8.8376e-05 - 716ms/epoch - 14ms/step
Epoch 194/300
50/50 - 1s - loss: 9.6428e-05 - mse: 9.6428e-05 - val_loss: 7.4754e-05 - val_mse: 7.4754e-05 - 543ms/epoch - 11ms/step
Epoch 195/300
50/50 - 1s - loss: 1.0866e-04 - mse: 1.0866e-04 - val_loss: 1.0195e-04 - val_mse: 1.0195e-04 - 669ms/epoch - 13ms/step
Epoch 196/300
50/50 - 1s - loss: 7.8945e-05 - mse: 7.8945e-05 - val_loss: 9.0216e-05 - val_mse: 9.0216e-05 - 580ms/epoch - 12ms/step
Epoch 197/300
50/50 - 1s - loss: 6.8142e-05 - mse: 6.8142e-05 - val_loss: 6.1437e-05 - val_mse: 6.1437e-05 - 660ms/epoch - 13ms/step
Epoch 198/300
50/50 - 1s - loss: 6.1873e-05 - mse: 6.1873e-05 - val_loss: 5.7352e-05 - val_mse: 5.7352e-05 - 712ms/epoch - 14ms/step
Epoch 199/300
50/50 - 1s - loss: 7.9765e-05 - mse: 7.9765e-05 - val_l

Epoch 254/300
50/50 - 1s - loss: 6.8297e-05 - mse: 6.8297e-05 - val_loss: 3.7333e-05 - val_mse: 3.7333e-05 - 860ms/epoch - 17ms/step
Epoch 255/300
50/50 - 1s - loss: 7.0618e-05 - mse: 7.0618e-05 - val_loss: 7.0432e-05 - val_mse: 7.0432e-05 - 712ms/epoch - 14ms/step
Epoch 256/300
50/50 - 1s - loss: 4.3550e-05 - mse: 4.3550e-05 - val_loss: 3.5983e-05 - val_mse: 3.5983e-05 - 588ms/epoch - 12ms/step
Epoch 257/300
50/50 - 1s - loss: 7.2338e-05 - mse: 7.2338e-05 - val_loss: 6.3611e-05 - val_mse: 6.3611e-05 - 547ms/epoch - 11ms/step
Epoch 258/300
50/50 - 1s - loss: 7.8460e-05 - mse: 7.8460e-05 - val_loss: 7.4838e-05 - val_mse: 7.4838e-05 - 897ms/epoch - 18ms/step
Epoch 259/300
50/50 - 1s - loss: 5.5034e-05 - mse: 5.5034e-05 - val_loss: 1.8882e-04 - val_mse: 1.8882e-04 - 761ms/epoch - 15ms/step
Epoch 260/300
50/50 - 1s - loss: 6.8296e-05 - mse: 6.8296e-05 - val_loss: 4.3918e-05 - val_mse: 4.3918e-05 - 558ms/epoch - 11ms/step
Epoch 261/300
50/50 - 1s - loss: 4.3709e-05 - mse: 4.3709e-05 - val_l

In [36]:
#use the test data to evaluate the model
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)
model.save('model_n.h5')

29/29 [==============================] - 0s 5ms/step - loss: 2.6348e-05 - mse: 2.6348e-05
[2.6347674065618776e-05, 2.634767588460818e-05]
76.83119201660156


In [24]:
#load the basic model trained before, one hidden layer with 32 ne
basic = load_model('basic.h5')

In [25]:
#use the test data to evaluate the model the generalization ability
loss_and_metrics = basic.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)


29/29 [==============================] - 0s 3ms/step - loss: 0.0067 - mse: 0.0067
[0.006697684060782194, 0.0066976845264434814]


In [26]:
#copy the basic model
basic_u1 = tf.keras.models.clone_model(basic)
basic_u1.set_weights(basic.get_weights())

inputs = tf.keras.Input(shape=(20,4))
basic_u1.layers[0].trainable = False
x = basic_u1.layers[0](inputs)
x1 = tf.keras.layers.SimpleRNN(32, activation='tanh', return_sequences=True)(x)
x2 = tf.keras.layers.Dense(2, activation='linear')(x1)

transf_model1 = tf.keras.Model(inputs=inputs,outputs=x2)

In [27]:
print(transf_model1.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 4)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 32)            1184      
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 20, 32)            2080      
                                                                 
 dense_4 (Dense)             (None, 20, 2)             66        
                                                                 
Total params: 3,330
Trainable params: 2,146
Non-trainable params: 1,184
_________________________________________________________________
None


In [28]:
transf_model1.compile(optimizer='adam', loss='mse', metrics=['mse'])
t0 = time.time()
history1 = transf_model1.fit(X_train, y_train, epochs=150, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

print(t1-t0)

Epoch 1/150
50/50 - 1s - loss: 0.1584 - mse: 0.1584 - val_loss: 0.0309 - val_mse: 0.0309 - 1s/epoch - 30ms/step
Epoch 2/150
50/50 - 0s - loss: 0.0187 - mse: 0.0187 - val_loss: 0.0113 - val_mse: 0.0113 - 449ms/epoch - 9ms/step
Epoch 3/150
50/50 - 0s - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0059 - val_mse: 0.0059 - 460ms/epoch - 9ms/step
Epoch 4/150
50/50 - 0s - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0039 - val_mse: 0.0039 - 468ms/epoch - 9ms/step
Epoch 5/150
50/50 - 0s - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0029 - val_mse: 0.0029 - 498ms/epoch - 10ms/step
Epoch 6/150
50/50 - 0s - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0024 - val_mse: 0.0024 - 452ms/epoch - 9ms/step
Epoch 7/150
50/50 - 1s - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0021 - val_mse: 0.0021 - 505ms/epoch - 10ms/step
Epoch 8/150
50/50 - 0s - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0018 - val_mse: 0.0018 - 476ms/epoch - 10ms/step
Epoch 9/150
50/50 - 0s - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0016 - val_mse: 0.001

Epoch 66/150
50/50 - 1s - loss: 1.6587e-04 - mse: 1.6587e-04 - val_loss: 1.6612e-04 - val_mse: 1.6612e-04 - 537ms/epoch - 11ms/step
Epoch 67/150
50/50 - 1s - loss: 1.6202e-04 - mse: 1.6202e-04 - val_loss: 1.5998e-04 - val_mse: 1.5998e-04 - 556ms/epoch - 11ms/step
Epoch 68/150
50/50 - 1s - loss: 1.5634e-04 - mse: 1.5634e-04 - val_loss: 1.5780e-04 - val_mse: 1.5780e-04 - 527ms/epoch - 11ms/step
Epoch 69/150
50/50 - 1s - loss: 1.6232e-04 - mse: 1.6232e-04 - val_loss: 2.0001e-04 - val_mse: 2.0001e-04 - 520ms/epoch - 10ms/step
Epoch 70/150
50/50 - 1s - loss: 1.5374e-04 - mse: 1.5374e-04 - val_loss: 1.5703e-04 - val_mse: 1.5703e-04 - 527ms/epoch - 11ms/step
Epoch 71/150
50/50 - 1s - loss: 1.5328e-04 - mse: 1.5328e-04 - val_loss: 1.5226e-04 - val_mse: 1.5226e-04 - 520ms/epoch - 10ms/step
Epoch 72/150
50/50 - 1s - loss: 1.4972e-04 - mse: 1.4972e-04 - val_loss: 1.9486e-04 - val_mse: 1.9486e-04 - 630ms/epoch - 13ms/step
Epoch 73/150
50/50 - 0s - loss: 1.5134e-04 - mse: 1.5134e-04 - val_loss: 1.5

Epoch 128/150
50/50 - 0s - loss: 8.5525e-05 - mse: 8.5525e-05 - val_loss: 6.6438e-05 - val_mse: 6.6438e-05 - 475ms/epoch - 10ms/step
Epoch 129/150
50/50 - 0s - loss: 6.3653e-05 - mse: 6.3653e-05 - val_loss: 6.6721e-05 - val_mse: 6.6721e-05 - 460ms/epoch - 9ms/step
Epoch 130/150
50/50 - 0s - loss: 8.2075e-05 - mse: 8.2075e-05 - val_loss: 6.2809e-05 - val_mse: 6.2809e-05 - 468ms/epoch - 9ms/step
Epoch 131/150
50/50 - 0s - loss: 7.3397e-05 - mse: 7.3397e-05 - val_loss: 7.3353e-05 - val_mse: 7.3353e-05 - 467ms/epoch - 9ms/step
Epoch 132/150
50/50 - 0s - loss: 7.5381e-05 - mse: 7.5381e-05 - val_loss: 7.6625e-05 - val_mse: 7.6625e-05 - 441ms/epoch - 9ms/step
Epoch 133/150
50/50 - 0s - loss: 6.1089e-05 - mse: 6.1089e-05 - val_loss: 6.0683e-05 - val_mse: 6.0683e-05 - 464ms/epoch - 9ms/step
Epoch 134/150
50/50 - 0s - loss: 6.1552e-05 - mse: 6.1552e-05 - val_loss: 7.0349e-05 - val_mse: 7.0349e-05 - 496ms/epoch - 10ms/step
Epoch 135/150
50/50 - 0s - loss: 6.8015e-05 - mse: 6.8015e-05 - val_loss: 

In [29]:
#use the test data to evaluate the model
loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)

29/29 [==============================] - 0s 5ms/step - loss: 5.4600e-05 - mse: 5.4600e-05
[5.460048851091415e-05, 5.460048123495653e-05]
76.40822458267212


In [30]:
#unfreeze the basic model and do the fine-tuning for the whole Model
transf_model1.layers[1].trainable = True
print(transf_model1.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 4)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 32)            1184      
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 20, 32)            2080      
                                                                 
 dense_4 (Dense)             (None, 20, 2)             66        
                                                                 
Total params: 3,330
Trainable params: 3,330
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
t0 = time.time()
history1 = transf_model1.fit(X_train, y_train, epochs=150, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

Epoch 1/150
50/50 - 0s - loss: 5.6416e-05 - mse: 5.6416e-05 - val_loss: 5.5063e-05 - val_mse: 5.5063e-05 - 490ms/epoch - 10ms/step
Epoch 2/150
50/50 - 0s - loss: 5.2320e-05 - mse: 5.2320e-05 - val_loss: 4.8848e-05 - val_mse: 4.8848e-05 - 452ms/epoch - 9ms/step
Epoch 3/150
50/50 - 0s - loss: 5.5371e-05 - mse: 5.5371e-05 - val_loss: 5.1654e-05 - val_mse: 5.1654e-05 - 480ms/epoch - 10ms/step
Epoch 4/150
50/50 - 1s - loss: 5.3291e-05 - mse: 5.3291e-05 - val_loss: 5.4794e-05 - val_mse: 5.4794e-05 - 549ms/epoch - 11ms/step
Epoch 5/150
50/50 - 1s - loss: 5.5309e-05 - mse: 5.5309e-05 - val_loss: 4.8845e-05 - val_mse: 4.8845e-05 - 504ms/epoch - 10ms/step
Epoch 6/150
50/50 - 1s - loss: 4.9578e-05 - mse: 4.9578e-05 - val_loss: 5.7134e-05 - val_mse: 5.7134e-05 - 506ms/epoch - 10ms/step
Epoch 7/150
50/50 - 0s - loss: 6.5292e-05 - mse: 6.5292e-05 - val_loss: 7.9241e-05 - val_mse: 7.9241e-05 - 469ms/epoch - 9ms/step
Epoch 8/150
50/50 - 0s - loss: 5.9703e-05 - mse: 5.9703e-05 - val_loss: 7.2235e-05 - 

Epoch 64/150
50/50 - 0s - loss: 5.0386e-05 - mse: 5.0386e-05 - val_loss: 3.4240e-05 - val_mse: 3.4240e-05 - 460ms/epoch - 9ms/step
Epoch 65/150
50/50 - 1s - loss: 3.7140e-05 - mse: 3.7140e-05 - val_loss: 4.0685e-05 - val_mse: 4.0685e-05 - 510ms/epoch - 10ms/step
Epoch 66/150
50/50 - 0s - loss: 4.0363e-05 - mse: 4.0363e-05 - val_loss: 5.6087e-05 - val_mse: 5.6087e-05 - 447ms/epoch - 9ms/step
Epoch 67/150
50/50 - 0s - loss: 3.7661e-05 - mse: 3.7661e-05 - val_loss: 3.5309e-05 - val_mse: 3.5309e-05 - 491ms/epoch - 10ms/step
Epoch 68/150
50/50 - 1s - loss: 3.8819e-05 - mse: 3.8819e-05 - val_loss: 4.3410e-05 - val_mse: 4.3410e-05 - 525ms/epoch - 10ms/step
Epoch 69/150
50/50 - 0s - loss: 3.8663e-05 - mse: 3.8663e-05 - val_loss: 3.9317e-05 - val_mse: 3.9317e-05 - 434ms/epoch - 9ms/step
Epoch 70/150
50/50 - 0s - loss: 4.5372e-05 - mse: 4.5372e-05 - val_loss: 5.0306e-05 - val_mse: 5.0306e-05 - 497ms/epoch - 10ms/step
Epoch 71/150
50/50 - 0s - loss: 3.8348e-05 - mse: 3.8348e-05 - val_loss: 3.9728

Epoch 126/150
50/50 - 0s - loss: 3.7372e-05 - mse: 3.7372e-05 - val_loss: 3.0120e-05 - val_mse: 3.0120e-05 - 484ms/epoch - 10ms/step
Epoch 127/150
50/50 - 0s - loss: 4.0611e-05 - mse: 4.0611e-05 - val_loss: 2.9869e-05 - val_mse: 2.9869e-05 - 465ms/epoch - 9ms/step
Epoch 128/150
50/50 - 1s - loss: 2.8503e-05 - mse: 2.8503e-05 - val_loss: 3.6572e-05 - val_mse: 3.6572e-05 - 505ms/epoch - 10ms/step
Epoch 129/150
50/50 - 0s - loss: 2.8601e-05 - mse: 2.8601e-05 - val_loss: 3.0906e-05 - val_mse: 3.0906e-05 - 480ms/epoch - 10ms/step
Epoch 130/150
50/50 - 1s - loss: 2.8011e-05 - mse: 2.8011e-05 - val_loss: 4.7698e-05 - val_mse: 4.7698e-05 - 512ms/epoch - 10ms/step
Epoch 131/150
50/50 - 1s - loss: 3.3813e-05 - mse: 3.3813e-05 - val_loss: 6.4762e-05 - val_mse: 6.4762e-05 - 550ms/epoch - 11ms/step
Epoch 132/150
50/50 - 1s - loss: 4.5893e-05 - mse: 4.5893e-05 - val_loss: 4.5055e-05 - val_mse: 4.5055e-05 - 533ms/epoch - 11ms/step
Epoch 133/150
50/50 - 1s - loss: 4.2284e-05 - mse: 4.2285e-05 - val_lo

In [32]:
print(t1 - t0) #memory the computation time

loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=128)
print(loss_and_metrics)

76.83119201660156
57/57 [==============================] - 0s 4ms/step - loss: 3.1441e-05 - mse: 3.1441e-05
[3.1440988095710054e-05, 3.1440988095710054e-05]


In [33]:
#calculate the input and the output

print(scaler_X.mean_)
print(scaler_X.scale_)

print(scaler_y.mean_)
print(scaler_y.scale_)

[-8.52255639e+00  2.15281955e-01 -1.94235589e+02 -1.18421053e-02]
[3.31629559e+01 7.19090574e-01 3.03416159e+05 2.12483895e+00]
[-8.50221738  0.21348508]
[34.50823792  0.72072879]


In [34]:
transf_model1.save('transfer_model1.h5')

In [35]:
loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

29/29 [==============================] - 0s 7ms/step - loss: 3.1441e-05 - mse: 3.1441e-05
[3.1440988095710054e-05, 3.1440988095710054e-05]
